<a href="https://colab.research.google.com/github/ambideXtrous9/Machine-Learning-Assignments/blob/main/214161006_214161011_Assignment_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EE 527 Machine Learning Lab
## Assignment 07
-----------------------------------------
214161006 Pathak Sanchit Vivekanand

214161011 Sushovan Saha

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.datasets import fetch_openml
import math

## Dataset

In [ ]:
mnist_data = fetch_openml('mnist_784', version=1)
print(mnist_data.keys())

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])


In [ ]:
X_train, y_train = mnist_data['data'], mnist_data['target'].to_numpy().astype(int)
print(y_train[0])
X_test = X_train.iloc[50002,:].to_numpy().reshape(1,-1)
y_test = y_train[50002]
print('Shape of X:', X_train.shape, '\n', 'Shape of y:', y_train.shape)
print('Shape of X_test:', X_test.shape, '\n', 'Shape of y:', y_test.shape)

5
Shape of X: (70000, 784) 
 Shape of y: (70000,)
Shape of X_test: (1, 784) 
 Shape of y: ()


In [ ]:
y_test

6

## PCA

In [ ]:
def imageExtractionPCA(df,y,m=50000, d=500):

    X = df.iloc[0:m].to_numpy()
    y = y[0:m]
       
    #PCA
    mu = np.mean(X,axis=0)
    z = X-mu.T
    C = np.cov(z.T)
    eigenvalue, eigenvector = np.linalg.eig(C)
    idx = eigenvalue.argsort()[::-1]   
    eigenvalue = eigenvalue[idx]
    eigenvector = eigenvector[:,idx]
    Q = eigenvector[:,0:d]
    
    #Dimensionality Reduction Transform
    v = np.matmul(z,Q)
    
    return X,y,Q,v,mu

## K-D Tree

In [ ]:
class Node:
    def __init__(self,v,dp,y):
        self.varmax_idx = np.argmax(np.var(v, axis = 0))
        self.medianmax = np.median(v[:,self.varmax_idx])
        self.data = v
        self.label = y 
        self.depth = dp
        self.left = None
        self.right = None

In [ ]:
def KDTree(root,MaxDepth,LTh):
    if root.depth == MaxDepth or root.data.shape[0] < LTh:
        return root
    else:
        keep = (root.data[:,root.varmax_idx]<root.medianmax)
        root.left =  KDTree(Node(root.data[keep,:],root.depth+1,root.label[keep]),MaxDepth,LTh)
        root.right = KDTree(Node(root.data[~keep,:],root.depth+1,root.label[~keep]),MaxDepth,LTh)
        return root

## Image Search

In [ ]:
def similarImgGrpSearch(root,X_test_pca):
    #If current node is not leaf node
    if root.left==None and root.right==None and root!=None:
        return root.data, root.label
    if root!=None:
        dim = root.varmax_idx
        med = root.medianmax
        if X_test_pca[:,dim]<med and root.left!=None:
            return similarImgGrpSearch(root.left, X_test_pca)
        elif root.right!=None:
            return similarImgGrpSearch(root.right, X_test_pca)

In [ ]:
def getTopKimages(IDB,LDB,X_test,k):
    L = []
    for i in range(IDB.shape[0]):
        dist = np.linalg.norm(IDB[i,:] - X_test)
        L.append(dist)
    L = np.array(L)
    idxs = np.argpartition(L, k)[:k]
    idxs = np.array(idxs)
    dfb = pd.DataFrame(IDB)
    dfb = dfb.iloc[idxs,:].to_numpy()
    yb = pd.DataFrame(LDB)
    yb = yb.iloc[idxs,:].to_numpy()
    return dfb,yb

In [ ]:
def PCA_KDTree(X,Y,X_test,m=50000,d=500,k=169,LTh=5,depth=8):
    X,y,Q,v,mu = imageExtractionPCA(X,Y,m,d)
    root = Node(v,0,y)
    root = KDTree(root,depth,LTh)
    X_test_pca = np.matmul(X_test-mu,Q)
    imagesInDB,labelInDB = similarImgGrpSearch(root, X_test_pca)
    topkimg,topklbl = getTopKimages(imagesInDB,labelInDB,X_test_pca,k)
    print("PRECISION = ",np.sum(topklbl==y_test)/k)

### Target feature sub-space Dimension(d) = 500
### K-D Tree Max Depth (depth) = 8
### Leaf Node data No (LTh) = 200
### Top k images = 169

In [ ]:
PCA_KDTree(X_train,y_train,X_test,50000,500,169,200,8)

PRECISION =  0.7455621301775148


### Target feature sub-space Dimension(d) = 200
### K-D Tree Max Depth (depth) = 5
### Leaf Node data No (LTh) = 150
### Top k images = 800

In [ ]:
PCA_KDTree(X_train,y_train,X_test,50000,200,800,150,5)

PRECISION =  0.69125


### Target feature sub-space Dimension(d) = 20
### K-D Tree Max Depth (depth) = 4
### Leaf Node data No (LTh) = 1500
### Top k images = 300

In [ ]:
PCA_KDTree(X_train,y_train,X_test,50000,20,300,1500,4)

PRECISION =  0.9833333333333333


### Target feature sub-space Dimension(d) = 100
### K-D Tree Max Depth (depth) = 10
### Leaf Node data No (LTh) = 5
### Top k images = 10

In [ ]:
PCA_KDTree(X_train,y_train,X_test,50000,100,10,5,10)

PRECISION =  1.0
